In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import os
import time
import argparse
from tensorflow.keras import Model, Sequential, metrics, optimizers # modify hyemin
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from tensorflow.keras.applications import VGG19
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D,\
                                    MaxPooling2D, UpSampling2D, GlobalAveragePooling2D,\
                                    Layer, Lambda, Flatten, Reshape, Conv2DTranspose,\
                                    Activation, LeakyReLU, Dropout, InputLayer, ReLU

from tensorflow.keras import initializers
from tqdm import tqdm_notebook, tqdm
import pandas as pd
import matplotlib.pyplot as plt
import math
from glob import glob
from PIL import Image
import csv
import shutil
tf.random.set_seed(1234)

In [ ]:
def random_aug(img) :
    lower=0.8
    upper=1.2
#     img = tf.image.random_contrast(img, lower, upper)
#     noise = tf.random.normal(shape=tf.shape(img), mean=5.0, stddev=1, dtype=tf.float32)
#     img = tf.add(img, noise)
#     mean = [0.5692, 0.5692, 0.5692]
#     std = [0.3469, 0.3469, 0.3469]
    
    img /= 255
#     img -= mean
#     img /= std
    return img
def test_norm(img) :
#     mean = [0.5692, 0.5692, 0.5692]
#     std = [0.3469, 0.3469, 0.3469]
    img /= 255
#     img -= mean
#     img /= std
    return img

In [ ]:
def load_dataset(batch_size=256):
#     epochs = 15
    IMG_HEIGHT = 64
    IMG_WIDTH = 64

    # data preparation
    #train_image_generator = IageDataGenerator(rescale=1./255) # Generator for our training data
#     train_dir = "/home/data/LGC/X_ray/trainset/oodd_anode/0701/train0707/"
#     val_dir = "/home/data/LGC/X_ray/trainset/oodd_anode/0701/test0707/"
    train_dir = "/home/data/LGC/Cylindrical/ooddset/crop/train"
    val_dir = "/home/data/LGC/Cylindrical/ooddset/crop/val"
    test_dir = "/home/data/LGC/Cylindrical/ooddset/crop/test"

    train_image_generator = ImageDataGenerator(preprocessing_function=random_aug, rotation_range=2, width_shift_range=1.0,\
                                   height_shift_range=1.0, brightness_range=(0.8,1.2), zoom_range=0.02,\
                                   fill_mode='nearest', horizontal_flip=True,\
                                   vertical_flip=True)
    
    test_image_generator = ImageDataGenerator(preprocessing_function=test_norm) # Generator for our validation data

    train_dataset = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                             directory=train_dir,
                                                             shuffle=True,
                                                             target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                             class_mode = 'categorical',
                                                             interpolation = 'bicubic'
                                                               )
    
    val_dataset = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                             directory=val_dir,
                                                             shuffle=False,
                                                             target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                             class_mode = 'categorical',
                                                             interpolation = 'bicubic'
                                                               )

    test_dataset = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                            directory=test_dir,
                                                            shuffle=False,
                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                            class_mode = 'categorical',
                                                            interpolation = 'bicubic'
                                                           )

    nb_classes = 2
    print('lgc ok') 
    return train_dataset, val_dataset, test_dataset

In [ ]:
kl = tf.keras.losses.KLDivergence()
def rel_perc_l1_loss(targets, recon_features, loss_type, train=True) :
    if train:
        rc_loss = 0

        for h1, h2 in zip(targets, recon_features):
            h1 = K.batch_flatten(h1)
            h2 = K.batch_flatten(h2)
            h1 = (h1 - np.mean(h1, axis=-1, keepdims=True)) / np.std(h1, axis=-1, keepdims=True)
            h2 = (h2 - np.mean(h2, axis=-1, keepdims=True)) / np.std(h2, axis=-1, keepdims=True)
            rc_loss = rc_loss + K.mean(K.mean(K.abs(h1-h2)/(K.abs(h1)+1e-7), axis=-1))
            
    else:
        rc_loss = []
        
        for h1, h2 in zip(targets, recon_features):
            h1 = K.batch_flatten(h1)
            h2 = K.batch_flatten(h2)
            h1 = (h1 - np.mean(h1, axis=-1, keepdims=True)) / np.std(h1, axis=-1, keepdims=True)
            h2 = (h2 - np.mean(h2, axis=-1, keepdims=True)) / np.std(h2, axis=-1, keepdims=True)
            rc_loss.append(K.mean(K.abs(h1-h2)/(K.abs(h1)+1e-7), axis=1))
        
    return rc_loss

def perceptual_loss(recon_features, targets, loss_type, train=True):
# #     for h1, h2, weight in zip(h1_list, h2_list, selected_pm_layer_weights):

    if train:
        rc_loss = 0

        for h1, h2 in zip(targets, recon_features):
            h1 = K.batch_flatten(h1)
            h2 = K.batch_flatten(h2)
            rc_loss = rc_loss + K.mean(K.square(h1 - h2), axis=-1)
#             rc_loss = rc_loss + K.mean(1-tf.keras.losses.cosine_similarity(h1, h2, axis=-1))
            
    else:
        rc_loss = []
        
        for h1, h2 in zip(targets, recon_features):
            h1 = K.batch_flatten(h1)
            h2 = K.batch_flatten(h2)
            rc_loss.append(K.mean(K.square(h1 - h2), axis=1))
#             rc_loss.append(K.mean(1-tf.keras.losses.cosine_similarity(h1, h2, axis=-1)))
#     rc_loss = 0 if train else []
#     for x, y in zip(x1, y1):
#         N, H, W, C = x.shape

#         x_vec = tf.transpose(x, perm=[0,3,1,2])
#         y_vec = tf.transpose(y, perm=[0,3,1,2])

#         y_mu = K.mean(y_vec, axis=[0, 2, 3], keepdims=True)

#         x_centered = x_vec - y_mu
#         y_centered = y_vec - y_mu

#         x_normalized = x_centered / tf.norm(x_centered, ord=2, axis=1, keepdims=True)
#         y_normalized = y_centered / tf.norm(y_centered, ord=2, axis=1, keepdims=True)

#         ############
#         x_normalized = tf.reshape(x_normalized, [N, C, -1])                                # (N, C, H*W)
#         y_normalized = tf.reshape(y_normalized, [N, C, -1])                                # (N, C, H*W)

#         x_normalized = tf.transpose(x_normalized, perm=[0,2,1])

#         cosine_sim = tf.matmul(x_normalized, y_normalized)
    
    ############
#         d = 1 - cosine_sim
    
#         if train :
#             rc_loss += K.mean(d)
#         else : 
#             rc_loss.append(K.mean(d))
        
    return rc_loss

def cosine_dist(x, y):
    N, H, W, C = x.shape
    
    x_vec = tf.transpose(x, perm=[0,3,1,2])
    y_vec = tf.transpose(y, perm=[0,3,1,2])
    
    y_mu = K.mean(y_vec, axis=[0, 2, 3], keepdims=True)
    
    x_centered = x_vec - y_mu
    y_centered = y_vec - y_mu
    
    x_normalized = x_centered / tf.norm(x_centered, ord=2, axis=1, keepdims=True)
    y_normalized = y_centered / tf.norm(y_centered, ord=2, axis=1, keepdims=True)

    ############
    x_normalized = tf.reshape(x_normalized, [N, C, -1])                                # (N, C, H*W)
    y_normalized = tf.reshape(y_normalized, [N, C, -1])                                # (N, C, H*W)
    
    ### TODO ### einsum 으로 bmm 대체 가능, 3D의 경우 matmul로 가능 -> Debug 필요
#     cosine_sim = torch.bmm(x_normalized.transpose(1, 2), y_normalized)

    x_normalized = tf.transpose(x_normalized, perm=[0,2,1])
    
    cosine_sim = tf.matmul(x_normalized, y_normalized)
    
    ############
    d = 1 - cosine_sim 
    
    return d

def l2_dist(x, y):
    N, H, W, C = x.shape
    
    x_vec = tf.transpose(x, perm=[0,3,1,2])
    y_vec = tf.transpose(y, perm=[0,3,1,2])

    x_vec = tf.reshape(x_vec, [N, C, -1])
    y_vec = tf.reshape(y_vec, [N, C, -1])
    
    ### TODO ###
#     x_s = torch.sum(x_vec ** 2, dim=1, keepdim=True)
#     y_s = torch.sum(y_vec ** 2, dim=1, keepdim=True)
    
    x_s = K.sum(x_vec ** 2, axis=1, keepdims=True)
    y_s = K.sum(y_vec ** 2, axis=1, keepdims=True)
    ############
    
    A = tf.transpose(y_vec, perm=[0,2,1]) @ x_vec
    B = tf.transpose(x_s, perm=[0,2,1])
    
    dist = y_s - 2 * A + B
    dist = tf.transpose(dist, perm=[0,2,1])
    dist = K.clip(dist, min_value=0., max_value=100000)
    return dist

def l1_dist(x, y):
    N, H, W, C = x.shape
#     print(x.shape)
    x_vec = tf.transpose(x, perm=[0,3,1,2])
    y_vec = tf.transpose(y, perm=[0,3,1,2])
#     print(x_vec.shape)
    x_vec = tf.reshape(x_vec, [N, C, -1])
    y_vec = tf.reshape(y_vec, [N, C, -1])
#     print(x_vec.shape)
    dist = tf.expand_dims(x_vec, axis=2) - tf.expand_dims(y_vec, axis=3)
#     dist = x_vec - y_vec
#     print(dist.shape)
#     dist = K.max(K.abs(dist), axis=1) + K.mean(K.abs(dist), axis=1)
    dist = K.sum(K.abs(dist), axis=1)
#     print(dist.shape)
    dist = tf.transpose(dist, perm=[0,2,1])
#     print(dist.shape)
    dist = K.clip(dist, min_value=0., max_value=100000)
    
    return dist

def contextual_loss(x1, y1, loss_type, h=0.1, train=True):
    cx_loss = 0 if train else []
    for x, y in zip(x1, y1):
        if loss_type == 'cl_cosine':
            d = cosine_dist(x, y)
        elif loss_type == 'cl_l2':
            d = l2_dist(x, y)
        else:
            d = l1_dist(x, y)
            
        d_min = K.min(d, axis=2, keepdims=True)        # (N, H*W, 1)
        # Eq (2)
        d_tilde = d / (d_min + 1e-5)
        # Eq(3)
        w = K.exp((1 - d_tilde) / h)
        # Eq(4)
        cx_ij = w / K.sum(w, axis=2, keepdims=True)       # (N, H*W, H*W)
        # Eq (1)
        cx = K.mean(K.max(cx_ij, axis=1), axis=1)  # (N, )
        if train:
            cx_loss += K.mean(-K.log(cx))
        else:
            cx_loss.append(-K.log(cx))    
        ############
    return cx_loss

def diff(test_dataset, model, loss_type, selected_pm_layers):
    diffs_total = [[] for _ in selected_pm_layers]
    labels =[] 
    diffs = []
    diffs2 = []
    kl_loss_list = []
    file_path = test_dataset.filepaths
    if loss_type == 'pl':
        crit = perceptual_loss
    elif loss_type == 'pl2' :
        crit = rel_perc_l1_loss
    else:
        crit = contextual_loss   

    for _ in tqdm(range(len(test_dataset))):
        image_batch, label_batch = test_dataset.next()
        
#         mean, log_var, latent_z, recon = model(image_batch, training=False)
        mean, log_var, latent_z, recon = model(image_batch, training=False)
#         feature_2, _, _ = model(recon, training=False)
#         for i in range(len(mean)) :
#             tmp_mean = tf.expand_dims(mean[i], 0)
#             tmp_log_var =  tf.expand_dims(log_var[i], 0)
        targets = model.classifier(image_batch)
        recon_features = model.classifier(recon)
        
#         temp = [x+y for (x,y) in zip(crit(recon_features, targets, train=False, loss_type=loss_type),\
#                                      crit(feature_2, feature_1, train=False, loss_type=loss_type))]
        temp = crit(recon_features, targets, train=False, loss_type=loss_type)
        
        for diffs_tmp, tmp in zip(diffs_total, temp):
            diffs_tmp.append(tmp) 
        diffs2.append((recon-image_batch))
        labels.extend(np.argmax(label_batch, axis=1).tolist())
    
    fig=plt.figure(figsize=(25, 25))
    for i, a in enumerate(image_batch) :
        fig.add_subplot(10, 10, 2*i+1)
        plt.imshow(a)
        fig.add_subplot(10, 10, 2*i+2)
        plt.imshow(recon[i])
    plt.show()
    
    diffs = [K.concatenate(i, axis=0) for i in diffs_total]
    
    return diffs, kl_loss_list, labels, file_path # modify hyemin
            # torch.cat이 list안에 있는 tesnor를 하나로 뭉쳐줌.

    
def testing(test_diff, kl_loss_list, label, file_path):
#     test_diff2 = tf.reshape(test_diffv2, [test_diffv2.shape[0], -1]).numpy()
    from collections import defaultdict
#     test_diff2 = test_diffv2.reshape(test_diffv2.shape[0], -1)
    ##### ToDo #####
    loss = 0
    for i in test_diff:
        loss += i
    ################
#     loss += kl_loss_list
#     loss2 = (test_diff2**2).sum(axis=1)
#     df = pd.DataFrame({'label':label, 'loss':loss})
#     df.to_csv('sample_result.csv')
    #print('loss shape {}'.format(loss.shape()))
    #print('loss2 shape {}'.format(loss2.shape()))
    #print('label {}'.format(label))
    print(len(label), len(loss))
    try :
        fprs, tprs, threshold = metrics.roc_curve(label, loss, pos_label=1) # loss는 batchsize만큼만 들어감
    #     fprs2,tprs2, threshold2 = metrics.roc_curve(label, loss2, pos_label=1) # 여기는 총 test set 개수가 들어감
    #     print('threshold {}'.format(threshold))
    #     print('threshold2 {}'.format(threshold2))
        auc, tnr_at_tpr95, th_loss = get_curve(loss,label,normal_label=0)
        print('threshold : ', th_loss)
        FN_path=''
        FP_path=''
        FP_metric = defaultdict(list)
        loss_count = 0 
    #     th_loss = 1.56
        for i_path, i_loss in zip(file_path, loss.numpy()):
    #         print(i_path, i_loss, th_loss)
            if i_loss >= th_loss and i_path.split('/')[-3]=='normal' :
                FN_path += (i_path+'\n\n')
                loss_count+=1
    #             writer.add_text('FN_images', i_path, FN_count)
            elif i_loss < th_loss and i_path.split('/')[-3]=='novel' :
                FP_path += (i_path+'\n\n')
                FP_metric[i_path.split('/')[-2]].append(i_path.split('/')[-1])
        print('과검 :',loss_count)
        for ng_name in FP_metric.keys() :
            print('미검 '+ng_name, len(FP_metric[ng_name]) )
        auc_ori = metrics.auc(fprs, tprs)
        auroc_odin = metrics.roc_auc_score(label, loss)
        aupr_in = metrics.average_precision_score(label,loss)
    except : 
        auc_ori, auc, tnr_at_tpr95, auroc_odin, aupr_in = 0, 0, 0, 0, 0
    val_loss = np.mean(loss)   
    return auc_ori, auc, tnr_at_tpr95, auroc_odin, aupr_in, val_loss


def normal_novel(loss, label, normal_label=1):
    loss = np.array(loss)
    label = np.array(label)
    check = (label == normal_label)
    normal_loss = loss[check]
    novel_loss = loss[~check]
    return normal_loss, novel_loss

def get_curve(loss, label, normal_label=1):
    normal, novel = normal_novel(loss, label, normal_label)
    tp, fp, tnr_at_tpr95 = [], [], []
    normal, novel = np.sort(normal)[::-1], np.sort(novel)[::-1] # 역순으로 변경
    end = np.max([np.max(normal), np.max(novel)])
    normal = end - normal # 추가 해야함
    novel = end - novel    # 추가 해야함
    start = np.min([np.min(normal), np.min(novel)])
    num_k, num_n = normal.shape[0], novel.shape[0]
    recall_k = 0.98
    pred_k_num = math.ceil(num_k*recall_k)
    threshold_loss = (end-normal[-pred_k_num] + end-normal[-pred_k_num-1])/2
    
    tp = -np.ones([num_k+num_n+1], dtype=int)
    fp = -np.ones([num_k+num_n+1], dtype=int)
    tp[0], fp[0] = num_k, num_n
    k, n = 0, 0
    for l in range(num_k+num_n):
        if k == num_k:
            tp[l+1:] = tp[l]
            fp[l+1:] = np.arange(fp[l]-1, -1, -1)
            break
        elif n == num_n:
            tp[l+1:] = np.arange(tp[l]-1, -1, -1)
            fp[l+1:] = fp[l]
            break
        else:
            if novel[n] < normal[k]:
                n += 1
                tp[l+1] = tp[l]
                fp[l+1] = fp[l] - 1
            else:
                k += 1
                tp[l+1] = tp[l] - 1
                fp[l+1] = fp[l]
    tpr95_pos = np.abs(tp / num_k - recall_k).argmin()
    tnr_at_tpr95 = 1. - fp[tpr95_pos] / num_n
    tpr = tp/num_k
    fpr = fp/num_n
    from sklearn import metrics
    auroc = metrics.auc(fpr, tpr)
    #aupr_in = metrics.average_precision_score(labels, scores)
    #aupr_out = metrics.average_precision_score(-1 * labels + 1, 1 - scores)
    return auroc, tnr_at_tpr95, threshold_loss

In [ ]:
class DFCVAE(tf.keras.Model):
    def __init__(self, latent_dim: int, net_type: str='conv', selected_pm_layers=[]):
        super(DFCVAE, self).__init__()
        self.latent_dim = latent_dim
        assert net_type in ['simple', 'conv']
#         inputs = tf.keras.Input(shape=(64, 64, 3))
        self.encoder_conv1 = Conv2D(32, 4, strides=(2, 2), padding="same", kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.encoder_bn1 = BatchNormalization(momentum=0.9, epsilon=1e-5)

        self.encoder_conv2 = Conv2D(64, 4, strides=(2, 2), padding="same", kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.encoder_bn2 = BatchNormalization(momentum=0.9, epsilon=1e-5)
        
        self.encoder_conv3 = Conv2D(128, 4, strides=(2, 2), padding="same", kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.encoder_bn3 = BatchNormalization(momentum=0.9, epsilon=1e-5)

        self.encoder_conv4 = Conv2D(256, 4, strides=(2, 2), padding="same", kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.encoder_bn4 = BatchNormalization(momentum=0.9, epsilon=1e-5)
        
#         self.encoder_conv5 = Conv2D(128, 4, strides=(2, 2), padding="same", kernel_initializer=init1,bias_initializer=initializers.Zeros())
#         self.encoder_bn5 = BatchNormalization(momentum=0.9, epsilon=1e-5)
        
#         self.encoder_conv6 = Conv2D(512, 4, strides=(2, 2), padding="same", kernel_initializer=init1,bias_initializer=initializers.Zeros())
#         self.encoder_bn6 = BatchNormalization(momentum=0.9, epsilon=1e-5)
    
        self.z_mean = Dense(self.latent_dim, name="z_mean")
        self.z_log_var = Dense(self.latent_dim, name="z_log_var")
        
        self.decoder_dense = Dense(4096)
        self.decoder_conv1 = Conv2D(128, 3, strides=1, padding='same', kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.decoder_bn1 = BatchNormalization(momentum=0.9, epsilon=1e-3)
        
        self.decoder_conv2 = Conv2D(64, 3, strides=1, padding='same', kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.decoder_bn2 = BatchNormalization(momentum=0.9, epsilon=1e-3)
        
        self.decoder_conv3 = Conv2D(32, 3, strides=1, padding='same', kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.decoder_bn3 = BatchNormalization(momentum=0.9, epsilon=1e-3)
        
#         self.decoder_conv4 = Conv2D(32, 3, strides=1, padding='same', kernel_initializer=init1,bias_initializer=initializers.Zeros())
#         self.decoder_bn4 = BatchNormalization(momentum=0.9, epsilon=1e-3)

#         self.decoder_conv5 = Conv2D(16, 3, strides=1, padding='same', kernel_initializer=init1,bias_initializer=initializers.Zeros())
#         self.decoder_bn5 = BatchNormalization(momentum=0.9, epsilon=1e-3)

        self.decoder_conv4 = Conv2D(3, 3, strides=1, padding='same', activation='sigmoid', kernel_initializer=init1,bias_initializer=initializers.Zeros())
        self.classifier = self.make_classifier(selected_pm_layers)
        
#         inputs = tf.keras.Input(shape=(64, 64, 3))
#         self.output = self.call(inputs)
#         self.dfc_vae = Model(inputs, self.output)
    
    def call(self, input_tensor, training=False):
        x = self.encoder_conv1(input_tensor)
        x = LeakyReLU(0.2)(x)
        x = self.encoder_bn1(x, training=training)
        
        x = self.encoder_conv2(x)
        x = LeakyReLU(0.2)(x)
        x = self.encoder_bn2(x, training=training)
        
        x = self.encoder_conv3(x)
        x = LeakyReLU(0.2)(x)
        x = self.encoder_bn3(x, training=training)
        feature1 = x
        
        x = self.encoder_conv4(x)
        x = LeakyReLU(0.2)(x)
        x = self.encoder_bn4(x, training=training)
        feature2 = x
        
#         x = self.encoder_conv5(x)
#         x = LeakyReLU(0.2)(x)
#         x = self.encoder_bn5(x, training=training)
        
#         x = self.encoder_conv6(x)
#         x = LeakyReLU(0.2)(x)
#         x = self.encoder_bn6(x, training=training)
        
        x = Flatten()(x)
        z_mean = self.z_mean(x)
        z_log_var = self.z_log_var(x)
        z = self.reparameterize(z_mean, z_log_var, training)
        feature3 = z
        
        x = self.decoder_dense(z)
        x = ReLU()(x)
        
        x = Reshape((4, 4, 256))(x)
        
        x = UpSampling2D((2, 2), interpolation='nearest')(x)
        x = self.decoder_conv1(x)
        x = LeakyReLU(0.2)(x)
        x = self.decoder_bn1(x, training=training)
        
        x = UpSampling2D((2, 2), interpolation='nearest')(x)
        x = self.decoder_conv2(x)
        x = LeakyReLU(0.2)(x)
        x = self.decoder_bn2(x, training=training)

        x = UpSampling2D((2, 2), interpolation='nearest')(x)
        x = self.decoder_conv3(x)
        x = LeakyReLU(0.2)(x)
        x = self.decoder_bn3(x, training=training)
        
#         x = UpSampling2D((2, 2), interpolation='nearest')(x)
#         x = self.decoder_conv4(x)
#         x = LeakyReLU(0.2)(x)
#         x = self.decoder_bn4(x, training=training)
        
#         x = UpSampling2D((2, 2), interpolation='nearest')(x)
#         x = self.decoder_conv5(x)
#         x = LeakyReLU(0.2)(x)
#         x = self.decoder_bn5(x, training=training)

        x = UpSampling2D((2, 2), interpolation='nearest')(x)
        x = self.decoder_conv4(x)

        return z_mean, z_log_var, z, x
    
    
    def make_classifier(self, selected_pm_layers):
        pm = VGG19(include_top=False, weights='imagenet')

        outputs = [pm.get_layer(l).output for l in selected_pm_layers]
        classifier = Model(pm.input, outputs)
#         classifier = vgg_pretrained
#         for layer in classifier.layers :
#             layer.trainable = False
    
        return classifier
    
    def reparameterize(self, mean, logvar, training):
        if training :
            eps = tf.random.normal(shape=tf.shape(mean))
        else :
            eps = tf.random.normal(shape=tf.shape(mean), seed=0)
        return eps * tf.exp(logvar * .5) + mean 

In [ ]:
loss_type = "cl_cosine"
latent_dim = 100
num_epochs = 200
lr = 0.001
batch_size = 32
log_dir = 'cylindrical_cl_cosine_crop_64'

In [ ]:
init1 = initializers.he_normal()
init2 = initializers.RandomNormal(stddev=0.015)

In [ ]:
selected_pm_layers = ['block4_conv1', 'block5_conv1']
selected_pm_layer_weights = [1.0, 1.0, 1.0]
model = DFCVAE(latent_dim, net_type='conv', selected_pm_layers=selected_pm_layers)
# model.classifier.trainable = False
print("LGC import Test #1")

In [ ]:
train_dataset, val_dataset, test_dataset = load_dataset(batch_size=batch_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr)
crit = contextual_loss
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
def get_negative_sample(img) :
    H, W, C = tf.shape(img)
    
    H, W = tf.cast(H, tf.float32), tf.cast(W, tf.float32)
    erase_w = tf.cast(tf.random.uniform([], 0.2*W, 0.3*W), tf.int32)
    erase_h = tf.cast(tf.random.uniform([], 0.2*H, 0.3*H), tf.int32)
    start_x = tf.cast(tf.random.uniform([], 0, 0.7*W), tf.int32)
    start_y = tf.cast(tf.random.uniform([], 0, 0.7*H), tf.int32)
    erase_area = tf.cast(tf.random.uniform([erase_h, erase_w, 1], 0, 255, tf.int32), tf.float32)/255.
    erase_area = tf.concat([erase_area]*3, axis=-1)
    
    output_img = np.copy(img)
    output_img[start_y:start_y+erase_h, start_x:start_x+erase_w, :] = erase_area
    return output_img

In [ ]:
def negative_sampling(inp_batch) :
    output = []
    for img in inp_batch :
        output.append(get_negative_sample(img))
    return np.array(output)

In [ ]:
kl = tf.keras.losses.KLDivergence()
best_tnr_tpr98 = 0
min_val_loss = 100
best_epoch = 0
for epoch in range(1, num_epochs + 1):
    t = time.time()
    kl_loss_list = []
    rc_loss_list = []
    rc_loss_list2 = []
    model.trainable=True
    model.classifier.trainable=False
    
    for _ in tqdm(range(len(train_dataset))):
        train_x = train_dataset.next()[0]
        negative_sample = negative_sampling(train_x)
        with tf.GradientTape() as tape:
#             mean, log_var, z, reconstruction = model(train_x, training=True)
            mean, log_var, z, reconstruction = model(train_x, training=True)
            _, _, _, reconstruction2 = model(negative_sample, training=True)
#             feature_2, _, _ = model(reconstruction, training=True)
#             noise = tf.random.normal(shape=tf.shape(train_x), mean=0, stddev=0.05, dtype=tf.float32)
            h1_list = model.classifier(train_x)
            h2_list = model.classifier(reconstruction)
            h3_list = model.classifier(negative_sample)
            h4_list = model.classifier(reconstruction2)
            kl_loss = kl(tf.fill([z.shape[0], z.shape[1]],1.0),tf.nn.softmax(z,axis=1)) / z.shape[1]
#             kl_loss = kl(tf.nn.softmax(z,axis=1),\
#                          tf.nn.softmax(tf.random.normal(shape=tf.shape(z), mean=0.0, stddev=0.00001),axis=1)) / z.shape[1]
#             kl_loss = 1 + log_var - tf.square(mean) - tf.exp(log_var)
#             kl_loss = tf.reduce_mean(kl_loss)
#             kl_loss *= -0.5
#             kl_loss = tf.reduce_mean(-0.5*tf.reduce_sum(1 + log_var - tf.square(mean) - tf.exp(log_var), 1))
#             rc_loss2 = tf.keras.losses.MSE(train_x, reconstruction)
#             rc_loss2 = crit(feature_2, feature_1, train=True, loss_type=loss_type)
            rc_loss1 = crit(h2_list, h1_list, train=True, loss_type=loss_type)
            rc_loss2 = crit(h4_list, h3_list, train=True, loss_type=loss_type)
            rc_loss3 = crit(h4_list, h1_list, train=True, loss_type=loss_type)
            rc_loss4 = crit(h2_list, h3_list, train=True, loss_type=loss_type)
#             rc_loss = (rc_loss1+rc_loss3) - 0.1/(rc_loss1+rc_loss3) * (rc_loss2+rc_loss4)
            rc_loss = (rc_loss1+rc_loss3) / (rc_loss2+rc_loss4)
            p2p_loss = K.mean((reconstruction-train_x)**2)
            total_loss = kl_loss + rc_loss
        grad = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grad, model.trainable_variables))

        kl_loss_list.append(kl_loss.numpy())
        rc_loss_list.append(rc_loss.numpy())
        rc_loss_list2.append(p2p_loss.numpy())
    kl_last_loss = np.mean(kl_loss_list)
    rc_last_loss = np.mean(rc_loss_list)
    rc_loss_list2 = np.mean(rc_loss_list2)
    
    fig=plt.figure(figsize=(25, 25))
    for i, a in enumerate(train_x) :
        fig.add_subplot(10, 10, 2*i+1)
        plt.imshow(a)
        fig.add_subplot(10, 10, 2*i+2)
        plt.imshow(reconstruction[i])
    plt.show()
    
    model.trainable=False
    print('Epoch {}, Loss1: {}, Loss2: {}, Loss3: {},  Remaining Time at Epochs: {:.2f}'.format(
            epoch, kl_last_loss, rc_last_loss, rc_loss_list2, time.time() - t
        ))
    test_diff, test_diff2, label, file_path = diff(val_dataset, model, loss_type, selected_pm_layers)
    auc_ori, auc, tnr_at_tpr95, auroc_odin, aupr_in, val_loss = testing(test_diff, test_diff2, label, file_path)
    if val_loss < min_val_loss :
        min_val_loss = val_loss
        best_epoch = epoch
    print(best_epoch)
    model.save("./results/"+log_dir+"_oodd-{:03d}".format(epoch))
    print('-------------validation_result--------------')
    print('auc_ori',auc_ori)
    print('auc',auc)
    print('tnr_at_tpr95',tnr_at_tpr95)
    print('aupr_in',aupr_in)
    print('val_loss',val_loss)

    test_diff, test_diff2, label, file_path = diff(test_dataset, model, loss_type, selected_pm_layers)
    auc_ori, auc, tnr_at_tpr95, auroc_odin, aupr_in, test_loss = testing(test_diff, test_diff2, label, file_path)
    print('-------------test_result--------------')
    print('auc_ori',auc_ori)
    print('auc',auc)
    print('tnr_at_tpr95',tnr_at_tpr95)
    print('aupr_in',aupr_in)
    print('test_loss',test_loss)
    print(best_epoch, min_val_loss)
    if best_tnr_tpr98 < tnr_at_tpr95 :
        best_tnr_tpr98 = tnr_at_tpr95
    
model.save("./results/"+log_dir+"_oodd_last")

In [ ]:
loaded_model = tf.keras.models.load_model('results/')

In [ ]:
test_dataset.reset()
tf.random.set_seed(1234)
np.random.seed(0)
test_diff, test_diff2, label, file_path = diff(test_dataset, loaded_model, loss_type, selected_pm_layers)
auc_ori, auc, tnr_at_tpr95, auroc_odin, aupr_in, val_loss = testing(test_diff, test_diff2, label, file_path)
print('auc_ori',auc_ori)
print('auc',auc)
print('tnr_at_tpr95',tnr_at_tpr95)
print('aupr_in',aupr_in)
print('val_loss',val_loss)

In [ ]:
from glob import glob
from PIL import Image
import csv
import shutil
import cv2
inference_set = glob("")
len(inference_set)

In [ ]:
tf.random.set_seed(1234)

count = 0 
loss_list = []
true_list = []
plt.figure(figsize = (20,20))
test_dataset.reset()
tf.random.set_seed(1234)
np.random.seed(0)
for img in tqdm_notebook(inference_set) :
    tmp_img = np.array(Image.open(img).convert('RGB').resize((64,64), Image.BICUBIC), dtype=np.float32)/255.
    tmp_img = np.expand_dims(tmp_img, 0)
    _, _, z, reconstruction = loaded_model(tmp_img, training=False)
    
    h1_list = loaded_model.classifier(tmp_img)
    h2_list = loaded_model.classifier(reconstruction)
    
    rc_loss = crit(h2_list, h1_list, train=False, loss_type=loss_type)
    loss = np.sum(rc_loss)
#     if loss >=1.67 and img.split('/')[-3]=='normal' :
    if loss>0.41 :
        pred = 'novel' 
    else :
        pred = 'normal'
    true = img.split('/')[-3]
    
#     if (true == 'normal') and 0.08 <= loss :
#     if  0.12 < loss <= 0.18 :
    loss_list.append(loss)
    true_list.append(true)
    count+=1
#         ax = plt.subplot(6,3,count)
#         plt.imshow(np.concatenate([tmp_img[0],reconstruction[0]], axis=1), cmap='gray')
#         ax.set_title((true,str(loss)))
#         plt.axis('off')
#         if count==12 : break
plt.show()
#         print(count, true, pred, loss, img.split('/')[-1])

In [ ]:
ok_loss = np.array(loss_list)[np.array(true_list)=='normal']
ng_loss = np.array(loss_list)[np.array(true_list)=='novel']
len(ok_loss), len(ng_loss)

In [ ]:
plt.hist(ok_loss, bins=20, density=True, alpha=0.7, histtype='stepfilled', color='skyblue')
plt.hist(ng_loss, bins=20, density=True, alpha=0.3, histtype='stepfilled', color='red')
plt.show()

In [ ]:
for img in np.array(inference_set)[(np.array(true_list)=='normal')&(np.array(loss_list)>0.5)] :
    plt.imshow(Image.open(img), cmap='gray')
    plt.show()